In [42]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

from torchvision import transforms
from torchvision.models.resnet import BasicBlock

# Класс для работы с датасетом изображений
class ImageDataset(Dataset):
    """
    Dataset для загрузки изображений и (опционально) меток классов.
    
    Аргументы:
        image_dir (str): Путь к директории с изображениями (JPG).
        label_file (str, optional): CSV-файл с метками. Должен содержать столбцы 'Id' и 'Category'.
        transform (callable, optional): Преобразования, применяемые к изображениям.
    """
    def __init__(self, image_dir, label_file=None, transform=None):
        self.image_dir = image_dir
        self.transform = transform

        # Загрузка меток, если указаны
        if label_file is not None:
            self.label_file = pd.read_csv(label_file)
            # Словарь соответствия: имя файла → категория
            self.labels = {
                row['Id']: row['Category'] 
                for _, row in self.label_file.iterrows()
            }
            self.has_labels = True
        else:
            self.labels = None
            self.has_labels = False

        # Список файлов изображений (только .jpg, отсортирован по имени)
        self.image_files = sorted([
            f for f in os.listdir(image_dir) 
            if f.endswith('.jpg')
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Загружаем изображение
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(image_path).convert("RGB")

        # Применяем трансформации, если заданы
        if self.transform:
            image = self.transform(image)

        # Возвращаем (изображение, метка) или только изображение
        if self.has_labels:
            label = self.labels[self.image_files[idx]]
            return image, label
        else:
            return image

# Аугментации и нормализация для обучающей выборки
train_transform = transforms.Compose([
    transforms.Resize((40, 40)),                      # Изменение размера изображения
    transforms.RandomHorizontalFlip(p=0.5),           # Случайное отражение по горизонтали
    transforms.RandomRotation(degrees=15),            # Случайный поворот на ±15°
    transforms.ToTensor(),                            # Преобразование в тензор
    transforms.Normalize(                             # Нормализация по каналам (ImageNet-совместимая)
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])


In [43]:
# Создание обучающего датасета и DataLoader'а
train_dataset = ImageDataset(
    image_dir='/kaggle/input/bhw-1-dl-2024-2025/bhw1/trainval',
    label_file='/kaggle/input/bhw-1-dl-2024-2025/bhw1/labels.csv',
    transform=train_transform
)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Кастомная реализация ResNet-34 (на основе BasicBlock)
class CustomResNet34(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet34, self).__init__()
        self.inplanes = 64

        # Начальный сверточный блок
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        # Четыре последовательных residual-слоя
        self.layer1 = self._make_layer(BasicBlock, 64, 3)
        self.layer2 = self._make_layer(BasicBlock, 128, 4, stride=2)
        self.layer3 = self._make_layer(BasicBlock, 256, 6, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 512, 3, stride=2)

        # Усреднение по пространственным координатам и классификатор
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        """
        Создание одного residual-слоя из нескольких блоков.
        """
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            # Понижение размерности (если нужно)
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        # Первый блок может изменять размерность
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion

        # Остальные блоки — обычные
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        """
        Прямой проход (forward) через всю сеть.
        """
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Определяем количество классов по CSV-файлу
num_classes = train_dataset.label_file['Category'].nunique()

# Инициализация модели, функции потерь и оптимизатора
model = CustomResNet34(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(
    model.parameters(),
    lr=0.01,
    momentum=0.9,
    weight_decay=1e-4
)


In [44]:
from tqdm import tqdm

# Параметры обучения
num_epochs = 11
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Перенос модели на GPU (если доступен)
model = model.to(device)

# Цикл обучения
for epoch in range(num_epochs):
    model.train()  # Переводим модель в режим обучения

    running_loss = 0.0
    correct = 0
    total = 0

    # Итерация по обучающему даталоадеру
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()        # Обнуляем градиенты
        outputs = model(inputs)      # Прямой проход
        loss = criterion(outputs, labels)  # Вычисляем loss
        loss.backward()              # Обратное распространение ошибки
        optimizer.step()             # Обновление весов

        running_loss += loss.item()

        # Вычисление точности
        _, predicted = torch.max(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Метрики за эпоху
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}] | "
          f"Loss: {epoch_loss:.4f} | "
          f"Accuracy: {epoch_accuracy:.2f}%")

# Сохраняем веса модели после завершения обучения
torch.save(model.state_dict(), 'model.pth')
print("✅ Модель сохранена в 'model.pth'")


100%|██████████| 3125/3125 [09:38<00:00,  5.40it/s]


Epoch [1/11], Loss: 4.7466, Accuracy: 4.88%


100%|██████████| 3125/3125 [09:34<00:00,  5.44it/s]


Epoch [2/11], Loss: 4.0685, Accuracy: 12.73%


100%|██████████| 3125/3125 [09:36<00:00,  5.42it/s]


...

Epoch [9/11], Loss: 2.4312, Accuracy: 41.06%


100%|██████████| 3125/3125 [09:23<00:00,  5.55it/s]


Epoch [10/11], Loss: 2.3058, Accuracy: 43.58%


100%|██████████| 3125/3125 [09:35<00:00,  5.43it/s]


Epoch [11/11], Loss: 2.2000, Accuracy: 45.80%


In [48]:
from torchvision import transforms

# Трансформации для тестовых изображений (без аугментаций)
test_transform = transforms.Compose([
    transforms.Resize((40, 40)),  # Приведение к нужному размеру
    transforms.ToTensor(),        # Перевод в тензор
    transforms.Normalize(         # Нормализация как в обучении
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Загрузка тестового датасета (без меток)
test_dir = '/kaggle/input/bhw-1-dl-2024-2025/bhw1/test'
test_dataset = ImageDataset(image_dir=test_dir, label_file=None, transform=test_transform)

# Перевод модели в режим инференса
model.eval()

# Списки для хранения предсказаний и имён файлов
predictions = []
image_ids = []

# Инференс без вычисления градиентов
with torch.no_grad():
    for idx in tqdm(range(len(test_dataset)), desc="Predicting"):
        image = test_dataset[idx]               # Получаем изображение
        image = image.unsqueeze(0).to(device)   # Добавляем батч-измерение и на GPU

        outputs = model(image)                  # Предсказание
        _, predicted = torch.max(outputs, 1)    # Выбор класса с наибольшей вероятностью

        predictions.append(predicted.item())
        image_ids.append(test_dataset.image_files[idx])

# Сохраняем результат в CSV-файл в формате Kaggle-сабмита
submission = pd.DataFrame({
    'Id': image_ids,
    'Category': predictions
})
submission.to_csv('labels_test.csv', index=False)
print("✅ Файл labels_test.csv успешно сохранён!")

100%|██████████| 10000/10000 [01:21<00:00, 123.32it/s]


In [47]:
# Переводим модель в режим обучения
model.train()

# Инициализация счетчиков
running_loss = 0.0
correct = 0
total = 0

# Обход всего обучающего набора
for inputs, labels in tqdm(train_loader, desc="Training"):
    inputs, labels = inputs.to(device), labels.to(device)

    # Обнуляем градиенты перед шагом оптимизации
    optimizer.zero_grad()

    # Прямой проход
    outputs = model(inputs)

    # Вычисляем loss
    loss = criterion(outputs, labels)

    # Обратный проход
    loss.backward()

    # Обновление параметров модели
    optimizer.step()

    # Суммируем loss
    running_loss += loss.item()

    # Предсказания: класс с максимальной вероятностью
    _, predicted = torch.max(outputs, 1)

    # Обновляем счётчики для точности
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

# Вычисляем точность за эпоху
accuracy = 100 * correct / total
print(f"Train Loss: {running_loss / len(train_loader):.4f} | Train Accuracy: {accuracy:.2f}%")


100%|██████████| 3125/3125 [09:34<00:00,  5.44it/s]
